# Multistage Stochastic Unit Commitment Problem

In [2]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg
from itertools import product


from sddip import config, utils, tree, parameters


## Parameters

In [3]:
test_case_name = "case6ww"

params = parameters.Parameters(test_case_name)

bus_df = params.bus_df
branch_df = params.bus_df
gen_df = params.bus_df
gen_cost_df = params.bus_df
ren_df = params.bus_df
storage_df = params.bus_df
scenario_df = params.scenario_df

In [4]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = params.gc
suc = params.suc
sdc = params.sdc
pg_min = params.pg_min
pg_max = params.pg_max
pl_max = params.pl_max

n_gens = params.n_gens

ptdf = params.ptdf

n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = params.gens_at_bus

rg_up_max = params.r_up
rg_down_max = params.r_down

min_up_time = params.min_up_time
min_down_time = params.min_down_time

penalty = params.penalty

n_storages = params.n_storages
storages_at_bus = params.storages_at_bus

rc_max = params.rc_max
rdc_max = params.rdc_max
soc_max = params.soc_max

eff_c = params.eff_c
eff_dc = params.eff_dc

    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_realizations_per_stage = params.n_realizations_per_stage
n_stages = params.n_stages

prob = params.prob
p_d = params.p_d
re = params.re

########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [5]:
# prob[t][n]
# Probability of realization n at stage t
#prob

In [6]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
#p_d

In [7]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
#ex_pd

In [8]:
scenario_tree = tree.ScenarioTree(n_realizations_per_stage)
print(scenario_tree)

ScenarioTree: Stages = 2, Nodes = 3


## Optimization

### Variables

In [9]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}
ys_charge = {}
ys_discharge = {}
u = {}
soc = {}


for t in range(n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        for s in range(n_storages):
            ys_charge[t,n,s] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_c_{t+1}_{n+1}_{s+1}")
            ys_discharge[t,n,s] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_d_{t+1}_{n+1}_{s+1}")
            u[t,n,s] = model.addVar(vtype = gp.GRB.BINARY, name = f"u_{t+1}_{n+1}_{s+1}")
            soc[t,n,s] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"soc_{t+1}_{n+1}_{s+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-02


### Constraints

In [10]:
# Objective
conditional_probabilities = []
p=1
for n in range(scenario_tree.n_stages):
    p = p*1/n_realizations_per_stage[n]
    conditional_probabilities.append(p)

obj = gp.quicksum(conditional_probabilities[t]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g]) 
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])
                    for g in range(n_gens)) \
        + gp.quicksum(conditional_probabilities[t]*penalty*(ys_p[t,n] + ys_n[t,n]) 
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t]))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n.index,g] for g in range(n_gens)) 
                    + gp.quicksum(eff_dc[s]*ys_discharge[t,n.index,s] - ys_charge[t,n.index,s] for s in range(n_storages)) 
                    + ys_p[t,n.index] - ys_n[t,n.index] 
                    == gp.quicksum(p_d[t][n.realization])-gp.quicksum(re[t][n.realization])
                    for t in range(n_stages)
                    for n in scenario_tree.get_stage_nodes(t)),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g]*x[t,n,g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g]*x[t,n,g]  
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-generation")


# Storage constraints
model.addConstrs((ys_charge[t,n,s] <= rc_max[s]*u[t,n,s]
                    for s in range(n_storages)
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-charge-rate")

model.addConstrs((ys_discharge[t,n,s] <= rdc_max[s]*(1-u[t,n,s])
                    for s in range(n_storages)
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-discharge-rate")

model.addConstrs((soc[t,n,s] <= soc_max[s]
                    for s in range(n_storages)
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-soc")

# SOC transfer
# t=0
soc_init = [0.5*s for s in soc_max]
model.addConstrs((soc[0,0,s] == soc_init[s] + eff_c[s]*ys_charge[0,0,s] - ys_discharge[0,0,s] 
                    for s in range(n_storages)),
                    "soc")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((soc[t,n,s] == soc[t-1, a_n, s] + eff_c[s]*ys_charge[t,n,s] - ys_discharge[t,n,s] 
                            for s in range(n_storages)),
                            "soc")


# Power flow constraints
for t in range(n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) 
                        + gp.quicksum(eff_dc[s]*ys_discharge[t,n,s] - ys_charge[t,n,s] for s in storages_at_bus[b]) 
                        - p_d[t][node.realization][b] + re[t][node.realization][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] == s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((x[t,n,g] - x[t-1,a_n,g] <= s_up[t,n,g] 
                        for g in range(n_gens)), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,a_n,g] == s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((y[t,n,g] - y[t-1,a_n,g] <= rg_up_max[g]
                        for g in range(n_gens)), 
                        "rate-up")
        model.addConstrs((y[t-1,a_n,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)), 
                        "rate-down")


# Minimum up- and down-time constraints
for g in range(n_gens):
    for t in range(1, min_up_time[g]):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors()
            model.addConstr((gp.quicksum(x[m.stage,m.index,g] for m in ancestors) >= (t+1)*s_down[t,n,g]), "min-uptime")
    
    for t in range(min_up_time[g], n_stages):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors(min_up_time[g])
            model.addConstr((gp.quicksum(x[m.stage, m.index,g] for m in ancestors) >= min_up_time[g]*s_down[t,n,g]), "min-uptime")

    for t in range(1, min_down_time[g]):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors()
            model.addConstr((gp.quicksum((1-x[m.stage,m.index,g]) for m in ancestors) >= (t+1)*s_up[t,n,g]), "min-downtime")

    for t in range(min_down_time[g], n_stages):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors(min_down_time[g])
            model.addConstr((gp.quicksum((1-x[m.stage,m.index,g]) for m in ancestors) >= min_down_time[g]*s_up[t,n,g]), "min-downtime")


model.update()
#model.display()

### Solve

In [11]:
model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")
print()
print(f"Optimal value: {obj.getValue()}")


Optimal value: 4366.252666581231


In [12]:
display_results = True

if display_results:
    x_out = [f"x[{t+1},{n+1},{g+1}]:  {x[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    y_out = [f"y[{t+1},{n+1},{g+1}]:  {y[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    s_up_out = [f"s_up[{t+1},{n+1},{g+1}]:  {s_up[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    s_down_out = [f"s_down[{t+1},{n+1},{g+1}]:  {s_down[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    soc_out = [f"soc[{t+1},{n+1},{s+1}]:  {soc[t,n,s].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for s in range(n_storages)]
    ys_charge_out = [f"y_c[{t+1},{n+1},{s+1}]:  {ys_charge[t,n,s].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for s in range(n_storages)]
    ys_discharge_out = [f"y_dc[{t+1},{n+1},{s+1}]:  {ys_discharge[t,n,s].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for s in range(n_storages)]
    ys_p_out =  [f"ys_p[{t+1},{n+1}]:  {ys_p[t,n].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t])]
    ys_n_out = [f"ys_n[{t+1},{n+1}]:  {ys_n[t,n].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t])]

    print("Commitment decisions")
    for text in x_out:
        print(f"{text}")

    print("Dispatch decisions")
    for text in y_out:
        print(f"{text}")

    print("Startup decisions")
    for text in s_up_out:
        print(f"{text}")

    print("Shutdown decisions")
    for text in s_down_out:
        print(f"{text}")

    print("State of charge")
    for text in soc_out:
        print(f"{text}")

    print("Charge/discharge")
    for text in ys_charge_out:
        print(f"{text}")
    for text in ys_discharge_out:
        print(f"{text}")

    print("Slack variables")
    for t1, t2 in zip(ys_p_out, ys_n_out):
        print(t1)
        print(t2)

Commitment decisions
x[1,1,1]:  -0.0
x[1,1,2]:  1.0
x[1,1,3]:  -0.0
x[2,1,1]:  -0.0
x[2,1,2]:  1.0
x[2,1,3]:  1.0
x[2,2,1]:  -0.0
x[2,2,2]:  1.0
x[2,2,3]:  1.0
Dispatch decisions
y[1,1,1]:  0.0
y[1,1,2]:  64.83258465044662
y[1,1,3]:  0.0
y[2,1,1]:  0.0
y[2,1,2]:  113.60427345450177
y[2,1,3]:  49.05931945854985
y[2,2,1]:  0.0
y[2,2,2]:  116.11899482013895
y[2,2,3]:  44.99999999999985
Startup decisions
s_up[1,1,1]:  -0.0
s_up[1,1,2]:  1.0
s_up[1,1,3]:  -0.0
s_up[2,1,1]:  0.0
s_up[2,1,2]:  0.0
s_up[2,1,3]:  1.0
s_up[2,2,1]:  0.0
s_up[2,2,2]:  -0.0
s_up[2,2,3]:  1.0
Shutdown decisions
s_down[1,1,1]:  0.0
s_down[1,1,2]:  0.0
s_down[1,1,3]:  0.0
s_down[2,1,1]:  0.0
s_down[2,1,2]:  0.0
s_down[2,1,3]:  0.0
s_down[2,2,1]:  0.0
s_down[2,2,2]:  0.0
s_down[2,2,3]:  0.0
State of charge
soc[1,1,1]:  5.0
soc[2,1,1]:  0.0
soc[2,2,1]:  0.0
Charge/discharge
y_c[1,1,1]:  0.0
y_c[2,1,1]:  0.0
y_c[2,2,1]:  0.0
y_dc[1,1,1]:  10.0
y_dc[2,1,1]:  5.0
y_dc[2,2,1]:  5.0
Slack variables
ys_p[1,1]:  0.0
ys_n[1,1]:

In [16]:
conditional_probabilities = []
p=1
for n in range(scenario_tree.n_stages):
    p = p*1/n_realizations_per_stage[n]
    conditional_probabilities.append(p)

costs = []
nodal_costs = []
c = 0
for t in reversed(range(n_stages)):
    c_n_list = []
    for n in range(scenario_tree.n_nodes_per_stage[t]):
        c_n = 0
        for g in range(n_gens):
            c_n += (y[t,n,g].x*gc[g] + s_up[t,n,g].x*suc[g] + s_down[t,n,g].x*sdc[g]+ penalty*(ys_p[t,n].x+ys_n[t,n].x))
        c += conditional_probabilities[t] * c_n
        c_n_list.append(c_n)
    nodal_costs.append(c_n_list)
    costs.append(c)


print(f"Optimal value function values: {costs}")
print(f"Nodal solutions: {nodal_costs}")

Optimal value function values: [2696.337569388166, 4366.252666581231]
Nodal solutions: [[2705.3325652998374, 2687.342573476494], [1669.9150971930649]]


In [14]:
model.setParam("OutputFlag", 1)
model.display()

Set parameter OutputFlag to value 1
Minimize
<gurobi.LinExpr: 11.669 y_1_1_1 + 1000.0 s_up_1_1_1 + 500.0 s_down_1_1_1
+ 10.333 y_1_1_2 + 1000.0 s_up_1_1_2 + 500.0 s_down_1_1_2 + 10.833 y_1_1_3
+ 1000.0 s_up_1_1_3 + 500.0 s_down_1_1_3 + 5000.0 ys_p_1_1 + 5000.0 ys_n_1_1
+ 5.8345 y_2_1_1 + 500.0 s_up_2_1_1 + 250.0 s_down_2_1_1 + 5.1665 y_2_1_2
+ 500.0 s_up_2_1_2 + 250.0 s_down_2_1_2 + 5.4165 y_2_1_3 + 500.0 s_up_2_1_3
+ 250.0 s_down_2_1_3 + 2500.0 ys_p_2_1 + 2500.0 ys_n_2_1 + 5.8345 y_2_2_1
+ 500.0 s_up_2_2_1 + 250.0 s_down_2_2_1 + 5.1665 y_2_2_2 + 500.0 s_up_2_2_2
+ 250.0 s_down_2_2_2 + 5.4165 y_2_2_3 + 500.0 s_up_2_2_3 + 250.0 s_down_2_2_3
+ 2500.0 ys_p_2_2 + 2500.0 ys_n_2_2>
Subject To
balance[0,<sddip.tree.Node object at 0x0000018423C02DC0>]: <gurobi.LinExpr: y_1_1_1 +
 y_1_1_2 + y_1_1_3 + -1.0 ys_c_1_1_1 + ys_d_1_1_1 + ys_p_1_1 + -1.0 ys_n_1_1> = 74.8326
balance[1,<sddip.tree.Node object at 0x0000018423C02C40>]: <gurobi.LinExpr: y_2_1_1 +
 y_2_1_2 + y_2_1_3 + -1.0 ys_c_2_1_1 + ys_d_